In [1]:
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import pandas as pd

In [2]:
idf_file = '../IDF/single_G.idf'
epw_file = '../weather/my_epw_new.epw'
building = ef.get_building(idf_file)
output_dir = './opt_out'

In [3]:
[materials.Name for materials in building.idfobjects["MATERIAL"]]# get a list of the Name property of all IDF objects of class MATERIAL

['Gypsum Plastering_.02',
 'Brick_.2',
 'Masonry - mediumweight Dry_.2',
 'Mortar_.03',
 'Ceramic/porcelain_.01',
 'Cement/plaster/mortar - cement plaster_.015',
 'Masonry - heavyweight Dry_.5',
 'Gypsum Plastering_.015',
 'Painted Oak_.035',
 'Masonry - heavyweight Dry_.12',
 'Masonry - heavyweight Dry_.09',
 'Brickwork Outer_.1',
 'XPS Extruded Polystyrene  - CO2 Blowing_.0795',
 'Concrete Block (Medium)_.1',
 'Gypsum Plastering_.013',
 'Masonry - heavyweight Dry_.27']

In [4]:
building.idfobjects["BUILDING"][0]


Building,
    Building,                 !- Name
    0,                        !- North Axis
    Suburbs,                  !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days

In [5]:
insulation = FieldSelector(class_name='Material',\
                           object_name='XXPS Extruded Polystyrene  - CO2 Blowing_.0795',\
                           field_name='Thickness')

In [6]:
zero_to_one_exclusive = CategoryParameter(options=[0.00001, 0.10, 0.20])

In [7]:
window_to_wall = wwr(RangeParameter(0.1,0.9))
window_to_wall
insulation_param = Parameter(selector=insulation,
                            value_descriptors=[zero_to_one_exclusive],
                            name='Insulation Thickness')

In [8]:
parameters = [window_to_wall,insulation_param]

In [9]:
objectives = ['Electricity:Facility','DistrictCooling:Facility'] # these get made into `MeterReader` or `VariableReader`
problem=EPProblem(parameters, objectives) # problem = parameters + objectives
problem

EPProblem(inputs=[Parameter(selector=wwrSelector(), value_descriptors=[RangeParameter(min=0.1, max=0.9)]), Parameter(selector=FieldSelector(field_name='Thickness', class_name='Material', object_name='XXPS Extruded Polystyrene  - CO2 Blowing_.0795'), value_descriptors=[CategoryParameter(options=[1e-05, 0.1, 0.2])])], outputs=[MeterReader(class_name='Output:Meter', func=<function sum_values at 0x7f5c613deee0>, key_name='Electricity:Facility'), MeterReader(class_name='Output:Meter', func=<function sum_values at 0x7f5c613deee0>, key_name='DistrictCooling:Facility')], minimize_outputs=[True, True], converters={'outputs': <class 'besos.objectives.MeterReader'>, 'constraints': <class 'besos.objectives.MeterReader'>})

In [10]:
samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=20)

#samples = pd.read_csv(filename+'.csv').drop(['Electricity:Facility','DistrictCooling:Facility'], axis=1)
samples

,"RangeParameter [0.1, 0.9]",Insulation Thickness
0,0.727941,0.00001
1,0.677218,0.20000
2,0.497429,0.10000
3,0.548317,0.00001
4,0.228612,0.20000
5,0.376945,0.10000
6,0.151318,0.20000
7,0.534769,0.00001
8,0.819946,0.10000
9,0.407971,0.10000


In [11]:
evaluator = EvaluatorEP(problem, building, out_dir=output_dir, err_dir=output_dir ,epw=epw_file) # evaluator = problem + building
outputs = evaluator.df_apply(samples, keep_input=True)
outputs.describe()

Executing:   0%|          | 0/20 [00:00<?, ?row/s]/home/rex/anaconda3/envs/building/lib/python3.8/site-packages/besos-2.1.3-py3.8.egg/besos/evaluator.py:190: UserWarning: for inputs: ['RangeParameter [0.1, 0.9]', 'Insulation Thickness'] problematic values were: RangeParameter [0.1, 0.9]    0.727941
Insulation Thickness         0.000010
Name: 0, dtype: float64
  warn(msg)


ValueError: The window is not aligned with the x or y axes